In [1]:
import os
import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load path to custom data

In [2]:
load_path = 'data/data4vgg.npz'
l = np.load(load_path)

print(l.files)

['trainlabel', 'trainimg', 'testimg', 'testlabel']


# Parse data

In [3]:
train_image = l['trainimg']
train_label = l['trainlabel']
test_image = l['testimg']
test_label = l['testlabel']
n_train = train_image.shape[0]
n_class = train_label.shape[1]
dimension = train_image.shape[1]
n_test = test_image.shape[0]

print ("%d train images loaded" % (n_train))
print ("%d test images loaded"  % (n_test))
print ("%d dimensional input"   % (dimension))
print ("%d classes"             % (n_class))
print ("shape of 'trainimg' is ", train_image.shape)
print ("shape of 'testimg' is ", test_image.shape)

69 train images loaded
18 test images loaded
37632 dimensional input
2 classes
shape of 'trainimg' is  (69, 37632)
shape of 'testimg' is  (18, 37632)


# Generate tensors for training and testing

In [4]:
train_image_tensor = np.ndarray((n_train, 112, 112, 3)) # 112 * 112 * 3 = 37632
for i in range(n_train):
    current_image = train_image[i, :]
#     print(current_image.shape)                          # Shape = (37632,)
    current_image = np.reshape(a=current_image, newshape=[112, 112, 3])
#     print(current_image.shape)                          # Shape_r = (112, 112, 3)
    train_image_tensor[i, :, :, :] = current_image
print('Shape of train_image_tensor is', train_image_tensor.shape)


test_image_tensor = np.ndarray((n_test, 112, 112, 3)) # 112 * 112 * 3 = 37632
for i in range(n_test):
    current_image = test_image[i, :]
#     print(current_image.shape)                          # Shape = (37632,)
    current_image = np.reshape(a=current_image, newshape=[112, 112, 3])
#     print(current_image.shape)                          # Shape_r = (112, 112, 3)
    test_image_tensor[i, :, :, :] = current_image
print('Shape of test_image_tensor is', test_image_tensor.shape)

Shape of train_image_tensor is (69, 112, 112, 3)
Shape of test_image_tensor is (18, 112, 112, 3)


# Define Functions to use pretrained VGG network

In [5]:
def conv_layer(x, weights, bias):
    conv = tf.nn.conv2d(x, filter=tf.constant(weights), strides=[1,1,1,1], padding='SAME')
    return tf.nn.bias_add(conv, bias)
    
def pool_layer(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
def preprocess(image, mean_pixel):
    return image - mean_pixel
    
def unprocess(image, mean_pixel):
    return image + mean_pixel

In [6]:
def vgg_net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )
    
    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(a=mean, axis=(0, 1))
    weights = data['layers'][0]
    net = {}
    current_image = input_image
    
    for i, name in enumerate(layers):
        kind = name[:4]
        
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            #matconvnet: weights are [width, height, in_channels, out_channels]
            #tensorflow: weights are [height, width, in_channels, out_channels]
            
            kernels = np.transpose(a=kernels, axes=(1, 0, 2, 3))
            bias = bias.reshape(-1)
            current_image = conv_layer(current_image, kernels, bias)
            
        elif kind == 'relu':
            current_image = tf.nn.relu(current_image)
            
        elif kind == 'pool':
            current_image = pool_layer(current_image)
        
        net[name] = current_image
        
    assert len(net) == len(layers)
    return net, mean_pixel

# Extract features from the VGG network

In [7]:
VGG_PATH = 'pre-trained-model/imagenet-vgg-verydeep-19.mat'

with tf.Graph().as_default(), tf.Session() as sess:
    image_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, 112, 112, 3])
    
    net_val, mean_pixel = vgg_net(data_path=VGG_PATH, input_image=image_placeholder)
    
    train_features = net_val['relu5_4'].eval(feed_dict={image_placeholder: train_image_tensor})
    test_features = net_val['relu5_4'].eval(feed_dict={image_placeholder: test_image_tensor})
    
print("TYPE OF 'train_features' is: ", type(train_features))
print("SHAPE OF 'train_features' is: ", train_features.shape)
print("TYPE OF 'test_features' is: " , type(test_features))
print("SHAPE OF 'test_features' is: ", test_features.shape)
print("PREPROCESSING DONE")

TYPE OF 'train_features' is:  <class 'numpy.ndarray'>
SHAPE OF 'train_features' is:  (69, 7, 7, 512)
TYPE OF 'test_features' is:  <class 'numpy.ndarray'>
SHAPE OF 'test_features' is:  (18, 7, 7, 512)
PREPROCESSING DONE


# Vectorize CNN features

In [8]:
train_vectorized = np.ndarray((n_train, 7*7*512))
test_vectorized = np.ndarray((n_test, 7*7*512))

for i in range(n_train):
    current_feature = train_features[i, :, :, :]
    current_feature_vec = np.reshape(a= current_feature, newshape=[1, -1])
    train_vectorized[i, :] = current_feature_vec

for i in range(n_test):
    current_feature = test_features[i, :, :, :]
    current_feature_vec = np.reshape(a= current_feature, newshape=[1, -1])
    test_vectorized[i, :] = current_feature_vec
    
print("SHAPE OF 'test_features' is: ", train_vectorized.shape)
print("SHAPE OF 'test_features' is: ", test_vectorized.shape)

SHAPE OF 'test_features' is:  (69, 25088)
SHAPE OF 'test_features' is:  (18, 25088)


# Define networks and functions(add 2 layers MLP)

In [9]:
#Params
learning_rate = 0.0001
epochs = 200
batch_size = 100
display_step = 10

#Network params
n_input = dimension
n_classes = n_class

#Weights and biases
weights = {
    'wd1': tf.Variable(tf.random_normal(shape=[7*7*512, 1024], stddev= 0.1)),
    'wd2': tf.Variable(tf.random_normal(shape=[1024, n_classes], stddev= 0.1))
}

biases = {
    'bd1': tf.Variable(tf.random_normal(shape=[1024], stddev= 0.1)),
    'bd2': tf.Variable(tf.random_normal(shape=[n_classes], stddev= 0.1))
}

#TF graph placeholder
x = tf.placeholder(dtype=tf.float32, shape=[None, 7*7*512])
y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(dtype=tf.float32)

In [10]:
def conv_basic(x, weights, biases, keep_prob):
    #Reshape input
    input_r = x
    
    #Vectorize
    dense = tf.reshape(input_r, shape=[-1, weights['wd1'].get_shape().as_list()[0]])
    
    #Fc1
    fc1 = tf.nn.relu(tf.add(tf.matmul(dense, weights['wd1']), biases['bd1']))
    fc1_dr = tf.nn.dropout(fc1, keep_prob=keep_prob)
    
    #Fc2
    out = tf.add(tf.matmul(fc1_dr, weights['wd2']), biases['bd2'])
    
    #Return everything
    out = {
        'input_r': input_r,
        'dense': dense,
        'fc1': fc1,
        'fc1_dr': fc1_dr,
        'out': out
    }
    
    return out

In [11]:
y_pred = conv_basic(x, weights, biases, keep_prob)['out']

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



# CNN FINETUNING

In [12]:
sess = tf.Session()
sess.run(init)

#training
for epoch in range(epochs):
    avg_loss = 0
    total_batch = int(n_train/batch_size)+1
    
    #Loop over all batches
    for i in range(total_batch):
        rand_idx = np.random.randint(n_train, size=batch_size)
        batch_x = train_vectorized[rand_idx, :]
        batch_y = train_label[rand_idx, :]
        
        #Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.7})
        
        #Compute avg loss
        avg_loss += sess.run(loss, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0}) / total_batch
        
    #Display logs per epoch step
    if epoch % display_step == 0:
        print('Epoch %03d/%03d, Loss = %04f' % (epoch, epochs, avg_loss))
        
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        test_accuracy = sess.run(accuracy, feed_dict={x: test_vectorized, y: test_label, keep_prob: 1.0})
        
        print('-->    Train Accuracy: ', train_accuracy)
        print('-->    Test Accuracy: ', test_accuracy)

Epoch 000/200, Loss = 1.732056
-->    Train Accuracy:  0.71
-->    Test Accuracy:  0.6111111
Epoch 010/200, Loss = 0.102751
-->    Train Accuracy:  0.95
-->    Test Accuracy:  0.7222222
Epoch 020/200, Loss = 0.212020
-->    Train Accuracy:  0.96
-->    Test Accuracy:  0.7222222
Epoch 030/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8888889
Epoch 040/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8333333
Epoch 050/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8333333
Epoch 060/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8888889
Epoch 070/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8888889
Epoch 080/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8888889
Epoch 090/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accuracy:  0.8888889
Epoch 100/200, Loss = 0.000000
-->    Train Accuracy:  1.0
-->    Test Accura